In [ ]:
from flask import Flask, request, render_template,jsonify
import urllib3, requests, json, os
import re
from PyDictionary import PyDictionary
from googletrans import Translator
from nltk.corpus import wordnet
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize
from textblob import TextBlob

l=[]
temp=[]
stop_words=[]
translator = Translator()

app = Flask(__name__)

@app.route('/sentimentanalysis', methods=['POST'])
def hello_world():
    name = request.form.get('name')    
    l=process(name)
    l=convert(l)
    manual_check(l)
    l=pre_process(l)
    l=final_filter(temp,l)
    result = sentiment_analysis(l)
    return render_template("result.html",result = result)

def process(text):
    regex = r'\b\w+\b'
    l=re.findall(regex,text)
    return l

def translate(i,n,line):
    translations = translator.translate([i], dest='en')
    for translation in translations:
        d=translation.text
        line[n]=d
           
def convert(line):
    for n,i in  enumerate(line):
        if not wordnet.synsets(i):
            translate(i,n,line)
        else:
            continue;
    return line

def manual_check(l):
    stop_words=['a','an','and','are','at','away','be','but','by','can','could','did','do','done','does','every','everybody','ever',
            'for','get','give','go','going','goods','got','he','had','have','has','having','his','himself','him','how'
           ,'i','I','if','in','it','itself','is','know','knew','knows','knowing','large','late','lately','later','man','me'
           ,'my','mom','men','might','may','member','myself','mr','ms','mrs','need','new','newer','newest','now','number','nothing'
           ,'of','on','old','once','other','others','own','part','parting','parts','per','present','presenting','presented','presents'
           ,'put','puts','quite','said','say','saw','same','second','seconds','seem','says','she','so','some','someone','somebody'
           ,'side','sides','something','somewhere','states','state','such','the','that','them','this','those','them','therefore'
           ,'think','thinks','thing','things','three','through','thus','today','together','tomorrow','to','u','under','until'
           ,'up','us','use','used','uses','we','was','want','wanted','which','where','what','when','while','who','with','within'
           ,'work','working','worked','would','works','you','yet','your','young','yours']
    for i in l:
        if i not in stop_words:
            temp.append(i)

def pre_process(l):
    process=""
    for i in l:
        process=process+i+" "
    process=process.strip()
    stop_words = set(stopwords.words('english'))

    word_tokens = word_tokenize(process)

    filtered_sentence = [w for w in word_tokens if not w in stop_words]

    filtered_sentence = []

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    return filtered_sentence

def final_filter(temp,l):
    res_list=[]
    flag=0;
    for i in temp:
        if((i not in l) and (i in stop_words)):
            res_list=temp.copy()
            flag=1
            break;
    if(flag==0):
           res_list=l.copy()
    return res_list

def sentiment_analysis(l):
    final_text=""
    with open('sentiment.txt', 'w') as filehandle:
        for listitem in l:
            filehandle.write('%s ' % listitem)
    lineList = [line.rstrip('\n') for line in open('sentiment.txt')]
    s=""
    for i in lineList:
        s=s+i;
    obj=TextBlob(s)
    sentiment=obj.sentiment.polarity
    return round(sentiment,2)*100

@app.route('/sentimentanalysis', methods = ['GET'])
def render_html():
    return render_template('home.html')
port = os.getenv('PORT', '5000')
if __name__ == "__main__":
    app.debug = False
    app.run(host='127.0.0.11', port=int(port))

 * Running on http://127.0.0.11:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Apr/2020 14:34:40] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [02/Apr/2020 14:34:40] "GET /sentimentanalysis HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2020 14:35:05] "POST /sentimentanalysis HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2020 14:35:53] "GET /sentimentanalysis HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2020 14:35:53] "GET / HTTP/1.1" 404 -
